In [3]:
# Now we will remove the signals from time t = 0 till t = 61 and on the right side extend till t = 1500. our data will now look like

voltage_2 = bat['voltage'][61:1000]
current_2 = bat['current'][61:1000]
time_2 = bat['time'][61:1000]

# Choosing the training points
rng = np.random.seed(1)
bias = 0.3 # setting bias to sample points from t = 61 to t = 136 
n_train_2 = 70
i = 0
training_indices_2 = np.zeros(n_train_2)
while i<n_train_2:
    if np.random.uniform() < bias:
        training_indices_2[i] = np.random.choice(np.squeeze(time_2[0:76]), size =  1) - 61
    else:
        training_indices_2[i] = np.random.choice(np.squeeze(time_2), size = 1) - 61
    i += 1

training_indices_2 = np.sort(training_indices_2).astype(int)

time_train_2, voltage_train_2 = time_2[training_indices_2], voltage_2[training_indices_2]
plt.scatter(time_train_2, voltage_train_2)    # training Points
plt.plot(time_train_2, voltage_train_2)
plt.plot(time_2, voltage_2, '--')
voltage_train_2.shape
voltage_train_2.shape - 61

# Testing data
n_test_2 = 500
testing_indices_2 = np.zeros(n_test_2)

i = 0
while i < n_test_2:
    dummy = np.random.choice(np.squeeze(time_2), size  = 1) - 61
    if dummy in training_indices_2:
        continue
    else:
        testing_indices_2[i] = dummy
        i += 1

testing_indices_2 = np.sort(testing_indices_2).astype(int)
time_test_2, voltage_test_2 = time_2[testing_indices_2].reshape(-1, 1), voltage_2[testing_indices_2].reshape(-1, 1)
# Testing data
plt.scatter(time_test_2, voltage_test_2)
plt.plot(time_test_2, voltage_test_2)
plt.plot(time_2, voltage_2, '--')

# Getting Prior with the test points
K_ss_2 = RBF_kernel(time_test_2, time_test_2, 2, 100)

voltage_prior_2 = np.random.multivariate_normal(mean = np.zeros(n_test_2), cov = K_ss_2, size = 3).T
#Plotting 5 sampled functions
plt.plot(time_test_2, voltage_prior_2)
plt.xlabel('time')
plt.ylabel('voltage')
plt.title('5 samples from GP prior')

# Applying Kernel to our training points
K_2 = RBF_kernel(time_train_2, time_train_2, 2, 100)
np.random.seed(1)
noise = np.random.randn() * 0.1
print(noise)
L_2 = np.linalg.cholesky(K_2 + noise * np.eye(n_train_2))
C_2 = np.linalg.inv(L_2)
K_inv_2 = np.dot(C_2.T, C_2)

# kernel comparing training and testing data
K_s_2 = RBF_kernel(time_train_2, time_test_2, 2, 100)

# Computing mean
mu_2 = (K_s_2.T @ K_inv_2 @ voltage_train_2).squeeze()

# computing Covariance
cov_2 = K_ss_2 - K_s_2.T @ K_inv_2 @ K_s_2

# computing standard deviation(sqrt of diagonal values of cov matrix)
std_dev_2 = np.sqrt(np.diag(cov_2)).squeeze()
# we draw samples from our posterior at our test points with mean = mu and covariance = cov
voltage_post_2 = np.random.multivariate_normal(mean = mu_2.squeeze(), cov = cov_2, size = 3).T

plt.figure(figsize = (8, 8))
plt.plot(time_2, voltage_2, '--r', label = 'Actual Data')
plt.plot(time_train_2, voltage_train_2, '*k', ms = 6, label = 'Observations')
plt.plot(time_test_2, voltage_post_2, label = 'Posteriors')
plt.plot(time_test_2, mu_2, 'm', label = 'predicted mean')
plt.gca().fill_between(time_test_2.flat, mu_2 - 2 * std_dev_2, mu_2 + 2 * std_dev_2, color = '#dddddd', label = '95% confidence region')
plt.legend()
plt.plot(time_2, voltage_2, '--b')
plt.plot(time_test_2, mu_2, 'm')

from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF
kernel = RBF(length_scale=1.0, length_scale_bounds=(1e-2, 1e2))
gaussian_process = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=9)
gaussian_process.fit(time_train_2, voltage_train_2)
gaussian_process.kernel_



NameError: name 'bat' is not defined